In [144]:
import pandas as pd
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline


import datetime
import glob


import nltk
import string, re

from sklearn.feature_extraction.text import TfidfVectorizer



stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()
wl = nltk.stem.WordNetLemmatizer()



In [145]:
covid_words = open("data/covid_words.txt").read().split()
religious_words = list(pd.read_csv('data/religious_words_final_df.csv').religious_words)

In [146]:
###########################################################
features = ['lang', 'created_at', 'original_text', 'clean_text', 'sentiment', 'location']
df = pd.DataFrame()

for fname in glob.glob(r"data/twitter_data/*.csv"):
    day_df=pd.read_csv(fname, low_memory=False)
    ### day_df['date']= datetime.date(2020, 5, int(fname.partition("May")[2][:-4]))
    df=df.append(day_df[features])
    print(fname, fname.partition("May")[2][:-4], len(df))

df = df[df.lang=='en'].drop('lang', axis=1)
df.to_csv('data/covid_data_full.csv', index=False)
###########################################################

data/twitter_data/covid_data_May24.csv 24 378386
data/twitter_data/covid_data_May3.csv 3 391293
data/twitter_data/covid_data_May10.csv 10 881233
data/twitter_data/covid_data_May17.csv 17 1292103


In [147]:
df = pd.read_csv('data/covid_data_full.csv', index_col=False, low_memory=False)
df.head(2)

,created_at,original_text,clean_text,sentiment,location
0,Sun May 24 23:59:59 +0000 2020,RT @morethanmySLE: Dr. Birx leaves the America...,RT morethanmySLE Dr. Birx leaves American peop...,"Sentiment(polarity=0.0, subjectivity=0.0)",NaN
1,Sun May 24 23:59:59 +0000 2020,RT @KaitSnod: There is no Corona Virus in Amer...,RT KaitSnod There Corona Virus America https /...,"Sentiment(polarity=0.0, subjectivity=0.0)",At the link down below 👀


In [148]:
def clean_twitter(text):
    text = "".join([char.lower() for char in text if char not in string.punctuation])
    tokens = re.split('\W+', text)
    ### text = [ps.stem(wl.lemmatize(word)) for word in tokens if word not in stopwords]
    text = [word for word in tokens if word not in stopwords]
#     for word in text.copy():
#         text.append(wl.lemmatize(word))
#         text.append(ps.stem(word))
        
    return " ".join(list(set(text)))

# clean_twitter("D@on't do that ! runn@er running!!!")

In [149]:
# df['clean_text_old'] = df['clean_text']
df['final_text'] = df['clean_text'].apply(lambda x: clean_twitter(x))
df.head(2)

,created_at,original_text,clean_text,sentiment,location,final_text
0,Sun May 24 23:59:59 +0000 2020,RT @morethanmySLE: Dr. Birx leaves the America...,RT morethanmySLE Dr. Birx leaves American peop...,"Sentiment(polarity=0.0, subjectivity=0.0)",NaN,rt leaves dr people covid19 answers birx moret...
1,Sun May 24 23:59:59 +0000 2020,RT @KaitSnod: There is no Corona Virus in Amer...,RT KaitSnod There Corona Virus America https /...,"Sentiment(polarity=0.0, subjectivity=0.0)",At the link down below 👀,rt virus corona kaitsnod tcoz9ndlkz3ml https a...


In [150]:
def find_religious_words(text):
########################################################################
###     return ", ".join(list(set(text.split()) & set(religious_words)))
########################################################################

    matched_words = list(filter(lambda x: x in text, religious_words))
    common_words = list()
    
    for idx_1, word_1 in enumerate(matched_words):
        for idx_2, word_2 in enumerate(matched_words[idx_1+1:]):
            if word_1 in word_2:
                common_words.append(word_1)

    final_matched_words = [x for x in matched_words if x not in set(common_words)]
    
    return ", ".join(final_matched_words)
                
find_religious_words('allah help me pls, amen! islam&peace ignore ram include ramadan')

'allah, amen, islam, ramadan'

In [151]:
df['religious_text'] = df['final_text'].apply(lambda x: find_religious_words(x))
df.head(2)

,created_at,original_text,clean_text,sentiment,location,final_text,religious_text
0,Sun May 24 23:59:59 +0000 2020,RT @morethanmySLE: Dr. Birx leaves the America...,RT morethanmySLE Dr. Birx leaves American peop...,"Sentiment(polarity=0.0, subjectivity=0.0)",NaN,rt leaves dr people covid19 answers birx moret...,
1,Sun May 24 23:59:59 +0000 2020,RT @KaitSnod: There is no Corona Virus in Amer...,RT KaitSnod There Corona Virus America https /...,"Sentiment(polarity=0.0, subjectivity=0.0)",At the link down below 👀,rt virus corona kaitsnod tcoz9ndlkz3ml https a...,


In [152]:
# df=df.loc[df['religious_text']!='']
df=df.sort_values('religious_text')
df.head(2)

,created_at,original_text,clean_text,sentiment,location,final_text,religious_text
0,Sun May 24 23:59:59 +0000 2020,RT @morethanmySLE: Dr. Birx leaves the America...,RT morethanmySLE Dr. Birx leaves American peop...,"Sentiment(polarity=0.0, subjectivity=0.0)",NaN,rt leaves dr people covid19 answers birx moret...,
828385,Sun May 10 05:19:14 +0000 2020,RT @ABsteward: NEW @JAMA_current\nInterpreting...,RT ABsteward NEW JAMA_current Interpreting Dia...,"Sentiment(polarity=0.13636363636363635, subjec...","ÜT: -2.149611,106.131112",rt tcoys7panhsp1 new diagnostic tests interpre...,


In [153]:
df.loc[df['religious_text']!='', 'is_religious_tweet'] = 'yes'
df.loc[df['religious_text']=='', 'is_religious_tweet'] = 'no'
df.head(2)

,created_at,original_text,clean_text,sentiment,location,final_text,religious_text,is_religious_tweet
0,Sun May 24 23:59:59 +0000 2020,RT @morethanmySLE: Dr. Birx leaves the America...,RT morethanmySLE Dr. Birx leaves American peop...,"Sentiment(polarity=0.0, subjectivity=0.0)",NaN,rt leaves dr people covid19 answers birx moret...,,no
828385,Sun May 10 05:19:14 +0000 2020,RT @ABsteward: NEW @JAMA_current\nInterpreting...,RT ABsteward NEW JAMA_current Interpreting Dia...,"Sentiment(polarity=0.13636363636363635, subjec...","ÜT: -2.149611,106.131112",rt tcoys7panhsp1 new diagnostic tests interpre...,,no


In [154]:
len(df[df.is_religious_tweet=='yes']), len(df[df.is_religious_tweet=='no'])

(242267, 1049830)

In [155]:
def parse_sentiment(sentiment, pattern):
    result = re.search(pattern, sentiment)
    return float(result.group(1))

df['polarity'] = df['sentiment'].apply(lambda x: parse_sentiment(x, 'Sentiment\(polarity=(.*), subjectivity=.*'))
df['subjectivity'] = df['sentiment'].apply(lambda x: parse_sentiment(x, '.*subjectivity=(.*)\)'))
df.head(2)

,created_at,original_text,clean_text,sentiment,location,final_text,religious_text,is_religious_tweet,polarity,subjectivity
0,Sun May 24 23:59:59 +0000 2020,RT @morethanmySLE: Dr. Birx leaves the America...,RT morethanmySLE Dr. Birx leaves American peop...,"Sentiment(polarity=0.0, subjectivity=0.0)",NaN,rt leaves dr people covid19 answers birx moret...,,no,0.000000,0.000000
828385,Sun May 10 05:19:14 +0000 2020,RT @ABsteward: NEW @JAMA_current\nInterpreting...,RT ABsteward NEW JAMA_current Interpreting Dia...,"Sentiment(polarity=0.13636363636363635, subjec...","ÜT: -2.149611,106.131112",rt tcoys7panhsp1 new diagnostic tests interpre...,,no,0.136364,0.454545


In [156]:
df['date'] = df['created_at'].apply(lambda x: pd.to_datetime(x[4:10]+' 2020', format="%b %d %Y"))
df.head(2)

,created_at,original_text,clean_text,sentiment,location,final_text,religious_text,is_religious_tweet,polarity,subjectivity,date
0,Sun May 24 23:59:59 +0000 2020,RT @morethanmySLE: Dr. Birx leaves the America...,RT morethanmySLE Dr. Birx leaves American peop...,"Sentiment(polarity=0.0, subjectivity=0.0)",NaN,rt leaves dr people covid19 answers birx moret...,,no,0.000000,0.000000,2020-05-24
828385,Sun May 10 05:19:14 +0000 2020,RT @ABsteward: NEW @JAMA_current\nInterpreting...,RT ABsteward NEW JAMA_current Interpreting Dia...,"Sentiment(polarity=0.13636363636363635, subjec...","ÜT: -2.149611,106.131112",rt tcoys7panhsp1 new diagnostic tests interpre...,,no,0.136364,0.454545,2020-05-10


In [157]:
df.to_csv('data/religious_twitter_df.csv', index=False)

In [158]:
df.iloc[:10000,].to_csv('data/religious_twitter_mini_df.csv', index=False)

In [159]:
religion_df=df.loc[df['religious_text']!='']
religion_df=religion_df.sort_values('religious_text')
religion_df.head(2)

,created_at,original_text,clean_text,sentiment,location,final_text,religious_text,is_religious_tweet,polarity,subjectivity,date
773298,Sun May 10 08:47:34 +0000 2020,Tom Hanks speech on ”The New World Order” and ...,Tom Hanks speech ” The New World Order ” after...,"Sentiment(polarity=0.13636363636363635, subjec...",Sverige,tom order new https ones speech chosen afterli...,afterlife,yes,0.136364,0.454545,2020-05-10
421701,Sun May 10 22:22:35 +0000 2020,You never know when you're gonna go! #Death #A...,You never know 're gon na go Death Afterlife G...,"Sentiment(polarity=0.0, subjectivity=0.0)","Watford, England",death na gon ghost know paranormal afterlife c...,afterlife,yes,0.000000,0.000000,2020-05-10


In [160]:
non_religion_df=df.loc[df['religious_text']=='']

In [161]:
print(df.shape, religion_df.shape, non_religion_df.shape)

(1292097, 11) (242267, 11) (1049830, 11)


######################### ------------------------------- #########################

In [ ]:
def clean_twitter(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(wl.lemmatize(word)) for word in tokens if word not in stopwords]
    return text

In [ ]:
def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text) - text.count(" ")), 3)*100

df['text_len'] = df['original_text'].apply(lambda x: len(x) - x.count(" "))
df['punct%'] = df['original_text'].apply(lambda x: count_punct(x))

In [ ]:
df.head(2)

In [ ]:
# tfidf_vect = TfidfVectorizer(analyzer=clean_text)
# tfidf_vect_fit = tfidf_vect.fit(df['original_text'])
# tfidf_data = tfidf_vect_fit.transform(df['original_text'])


# vect_df = pd.concat([df[['text_len', 'punct%']].reset_index(drop=True), pd.DataFrame(tfidf_data.toarray())], axis=1)
# vect_df.head(2)